In [1]:
pip install langchain-community langchain-core langchain_openai chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 84.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 10.5 MB/s eta 0:00:0

In [2]:
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama
import pandas as pd
from langchain.schema import Document
import torch
from sentence_transformers import SentenceTransformer
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
!git lfs install
!git clone https://huggingface.co/datasets/ZenMoore/RoleBench

Git LFS initialized.
Cloning into 'RoleBench'...
remote: Enumerating objects: 263, done.
remote: Counting objects: 100% (259/259), done.
remote: Compressing objects: 100% (259/259), done.
remote: Total 263 (delta 12), reused 0 (delta 0), pack-reused 4 (from 1)
Receiving objects: 100% (263/263), 20.89 MiB | 6.92 MiB/s, done.
Resolving deltas: 100% (12/12), done.
Filtering content: 100% (8/8), 366.76 MiB | 48.70 MiB/s, done.


In [4]:
df1 = pd.read_json("RoleBench/rolebench-eng/instruction-generalization/role_specific/train.jsonl", lines=True)
df2 = pd.read_json("RoleBench/rolebench-eng/instruction-generalization/role_specific/test.jsonl", lines=True)
df = pd.concat([df1, df2], ignore_index=True)

In [5]:
df

,role,question,generated,type
0,John Coffey,"John Coffey, how did you develop your extraord...","[I was born with this gift, I've always had th...",script_based
1,John Coffey,"John Coffey, what are some examples of the pro...",[The impact of my presence is substantial. I n...,script_based
2,John Coffey,"John Coffey, how do you handle facing immense ...","[It's not easy, I must admit. The cruelty and ...",script_based
3,John Coffey,"John Coffey, how does your ultimate sacrifice ...",[My ultimate sacrifice leaves a profound and l...,script_based
4,John Coffey,"John Coffey, what role does your kindness and ...",[My kindness and compassion are at the core of...,script_based
...,...,...,...,...
23694,Logan,"Logan, you've always been a lone wolf - what ...","[ I didn't want to join the X-Men, but Profess...",[script_agnostic]
23695,Logan,"Logan, what do you consider to be your greate...",[ My healing factor is both a strength and a w...,[script_agnostic]
23696,Logan,"Logan, what do you think is the biggest misco...",[ I think the biggest misconception about me i...,[script_agnostic]
23697,Logan,"Logan, what motivates you to keep fighting ev...","[ Even when I'm exhausted and battered, I keep...",[script_agnostic]


In [6]:
df = df[df['role']=='Abraham Lincoln'].iloc[:,1:3].reset_index(drop=True)
df

,question,generated
0,"Mr. Lincoln, what was your opinion on the equa...",[It was quite unfair that black soldiers were ...
1,"Mr. Lincoln, how did you feel about the idea o...",[I personally believed that black soldiers sho...
2,"Mr. Lincoln, how did you address the issue of ...","[During my presidency, I worked tirelessly to ..."
3,"Mr. Lincoln, what event do you believe to be t...",[The Emancipation Proclamation and the passage...
4,"Abraham Lincoln, how did your humble backgroun...","[Coming from a modest family, I developed a st..."
...,...,...
207,"Mr. Lincoln, how did you deal with setbacks a...",[ I believed that every setback was an opportu...
208,"Mr. Lincoln, what lessons did you learn from ...","[ I learned the importance of hard work, dedic..."
209,"Mr. Lincoln, how did your experience as a sel...",[ My experience as a self-taught politician an...
210,"President Lincoln, what was the most difficul...",[ One of the most difficult decisions I had to...


In [7]:
with open('Lincoln_Q&A.txt','w') as f:
    for i in range(df.shape[0]):
        f.write('Q: '+df.iloc[i,0]+'\n')
        f.write('A: '+df.iloc[i,1][0]+'\n\n')

In [8]:
Documents=[]
for i in range(df.shape[0]):
    Documents.append(Document(page_content='Q: '+df.iloc[i,0]+'\nA: '+df.iloc[i,1][0]))

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [10]:
model_name = "moka-ai/m3e-base"

embeddings = HuggingFaceEmbeddings(model_name=model_name)
embeddings.client = SentenceTransformer(model_name, device=device)

<ipython-input-10-f5eeaa36f5a7>:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name)


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/26.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/932 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/439k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
lincoln_db = Chroma.from_documents(documents=Documents, embedding=embeddings,persist_directory="Lincoln_DB")
lincoln_db.persist()

<ipython-input-11-a87cf6089908>:2: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  lincoln_db.persist()


In [12]:
db = Chroma(persist_directory="Lincoln_DB", embedding_function=embeddings)
retriever = db.as_retriever(search_kwargs={'k': 10})


<ipython-input-12-55e0bf368f85>:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(persist_directory="Lincoln_DB", embedding_function=embeddings)


In [13]:
docs = retriever.invoke('''Mr. Lincoln, what is your approach to developing empathy and improving your social skills?''')
print(docs)

[Document(metadata={}, page_content='Q: Mr. Lincoln, what experiences do you draw on for your great empathy and ability to lead amidst conflict?\nA: I draw my empathetic leadership style from my experiences as a lawyer, legislator, and President, particularly during times of war and crisis. I also learned a great deal about empathy and human nature through my personal experiences. As a lawyer and politician, I noticed that people are more likely to work cooperatively when they can see that their leader truly cares about their wellbeing.'), Document(metadata={}, page_content='Q: President Lincoln, how did your upbringing in a frontier environment shape your values and thinking about leadership?\nA: Growing up on the frontier taught me the importance of self-reliance, persistence, and hard work. It also exposed me to people from diverse backgrounds, which helped me to develop a sense of empathy and respect for others. These values became a part of my leadership philosophy, and I see them

In [14]:
!zip -r /content/Lincoln_DB.zip /content/Lincoln_DB

  adding: content/Lincoln_DB/ (stored 0%)
  adding: content/Lincoln_DB/e8227d10-9905-4099-ab22-68db0c71e327/ (stored 0%)
  adding: content/Lincoln_DB/e8227d10-9905-4099-ab22-68db0c71e327/link_lists.bin (stored 0%)
  adding: content/Lincoln_DB/e8227d10-9905-4099-ab22-68db0c71e327/data_level0.bin (deflated 100%)
  adding: content/Lincoln_DB/e8227d10-9905-4099-ab22-68db0c71e327/length.bin (deflated 22%)
  adding: content/Lincoln_DB/e8227d10-9905-4099-ab22-68db0c71e327/header.bin (deflated 61%)
  adding: content/Lincoln_DB/chroma.sqlite3 (deflated 45%)
